# Iteration 3: Sample length of stay on the ASU

## Plain English Summary

In this iteration we fix the sampling from lognormal distributions.  We create a function that converts the mean and standard deviation of the lognormal to the equivalent parameters of the underlying normal distribution.  This is because `numpy` requires the parameters to be in this format.



## Notebook imports

In [1]:
from IPython.display import Markdown, display
import math

## LLM Prompt

In [2]:
display(Markdown("inputs/04_iteration.md"))

### Main command

modify all `acute_treatment `functions to call a function call called `normal_moments_from_lognormal`. The function should accept the mean and standard deviation of the Lognormal distribution and return the mu, sigma of the underlying Normal distribution.  These returned values are the correct values to use when sampling from the Lognormal distributions. 

Show all of the model code: this includes all patient generator functions in the ASU class.

## Date and time of input to LLM

> 1530 2nd March 2024

## Observations on the LLM Output

* `normal_moments_from_lognormal` is defined twice. Once as function and once as part of the ASU class.


## Sources reported by the LLM

* None given - Perplexity appears to have stopped providing them.

## LLM Output

In [4]:
import simpy
import numpy as np

# External function to convert Lognormal moments to Normal moments
def normal_moments_from_lognormal(log_mean, log_std):
    phi = np.sqrt(log_std ** 2 + log_mean ** 2)
    mu = np.log(log_mean ** 2 / np.sqrt(log_std ** 2 + log_mean ** 2))
    sigma = np.sqrt(np.log(phi ** 2 / log_mean ** 2))
    return mu, sigma

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_acute_treatment(self, patient_type):
        if patient_type == "Rehab":
            mu, sigma = normal_moments_from_lognormal(np.log(7.4), np.log(8.6))
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        elif patient_type == "ESD":
            mu, sigma = normal_moments_from_lognormal(np.log(4.6), np.log(4.8))
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        else:
            mu, sigma = normal_moments_from_lognormal(np.log(7.0), np.log(8.7))
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        
        yield self.env.timeout(length_of_stay)
        print(f"Stroke patient {self.patient_count} finished treatment at {self.env.now} days")

    def tia_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(np.log(1.8), np.log(5.0))
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"TIA patient {self.patient_count} finished treatment at {self.env.now} days")

    def neuro_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(np.log(4.0), np.log(5.0))
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"Complex Neuro patient {self.patient_count} finished treatment at {self.env.now} days")

    def other_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(np.log(3.8), np.log(5.2))
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"Other patient {self.patient_count} finished treatment at {self.env.now} days")

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            self.env.process(self.stroke_acute_treatment(post_asu_destination))

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.tia_acute_treatment())

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.neuro_acute_treatment())

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.other_acute_treatment())

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)



Other patient 1 arrived at 0.682299246626044 days
Stroke patient 2 arrived at 0.758492494096446 days and will go to Rehab
Complex Neuro patient 3 arrived at 1.7197251681787575 days
Complex Neuro patient 4 arrived at 1.7239124716517824 days
Other patient 5 arrived at 1.8428056330127036 days
Complex Neuro patient 5 finished treatment at 2.5438416233957506 days
Complex Neuro patient 5 finished treatment at 2.574579148484992 days
Other patient 5 finished treatment at 2.6110293110407303 days
Other patient 6 arrived at 2.667297283001268 days
Other patient 7 arrived at 2.829483024580333 days
Other patient 8 arrived at 2.8669728469696336 days
Other patient 8 finished treatment at 2.871895250050423 days
Other patient 8 finished treatment at 2.9030957507835766 days
Other patient 8 finished treatment at 3.0632319772445014 days
Stroke patient 9 arrived at 3.1244783913299865 days and will go to ESD
Other patient 9 finished treatment at 3.455657557404602 days
Stroke patient 10 arrived at 3.801871241

## Testing log normal snippets

* Test of the lognormal sampling.  The code below is our own verified code. The source is given in the docstring.

In [3]:
import simpy
import numpy as np

# Define the normal moments calculation function
def normal_moments_from_lognormal(log_mean, log_std):
    phi = np.sqrt(log_std ** 2 + log_mean ** 2)
    mu = np.log(log_mean ** 2 / np.sqrt(log_std ** 2 + log_mean ** 2))
    sigma = np.sqrt(np.log(phi ** 2 / log_mean ** 2))
    return mu, sigma

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def normal_moments_from_lognormal(self, log_mean, log_std):
        phi = np.sqrt(log_std ** 2 + log_mean ** 2)
        mu = np.log(log_mean ** 2 / np.sqrt(log_std ** 2 + log_mean ** 2))
        sigma = np.sqrt(np.log(phi ** 2 / log_mean ** 2))
        return mu, sigma

    def stroke_acute_treatment(self, patient_type):
        if patient_type == "Rehab":
            mu, sigma = self.normal_moments_from_lognormal(np.log(7.4), np.log(8.6))
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        elif patient_type == "ESD":
            mu, sigma = self.normal_moments_from_lognormal(np.log(4.6), np.log(4.8))
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        else:
            mu, sigma = self.normal_moments_from_lognormal(np.log(7.0), np.log(8.7))
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        
        yield self.env.timeout(length_of_stay)
        print(f"Stroke patient {self.patient_count} finished treatment at {self.env.now} days")

    def tia_acute_treatment(self):
        mu, sigma = self.normal_moments_from_lognormal(np.log(1.8), np.log(5.0))
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"TIA patient {self.patient_count} finished treatment at {self.env.now} days")

    def neuro_acute_treatment(self):
        mu, sigma = self.normal_moments_from_lognormal(np.log(4.0), np.log(5.0))
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"Complex Neuro patient {self.patient_count} finished treatment at {self.env.now} days")

    def other_acute_treatment(self):
        mu, sigma = self.normal_moments_from_lognormal(np.log(3.8), np.log(5.2))
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"Other patient {self.patient_count} finished treatment at {self.env.now} days")

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            self.env.process(self.stroke_acute_treatment(post_asu_destination))

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.tia_acute_treatment())

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.neuro_acute_treatment())

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.other_acute_treatment())

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)



Complex Neuro patient 1 arrived at 0.3185658119016402 days
Other patient 2 arrived at 0.5630606404701978 days
Other patient 2 finished treatment at 0.841538435866932 days
Stroke patient 3 arrived at 1.1222349942868777 days and will go to Other
Stroke patient 4 arrived at 1.228678687145576 days and will go to Other
Stroke patient 5 arrived at 1.4747894048363512 days and will go to Other
Complex Neuro patient 6 arrived at 1.516451231082894 days
Stroke patient 6 finished treatment at 2.1669321406817788 days
Complex Neuro patient 6 finished treatment at 2.2047398010837354 days
Stroke patient 6 finished treatment at 2.2132420314199703 days
Stroke patient 6 finished treatment at 3.7743708813670453 days
Stroke patient 7 arrived at 4.7169735709915805 days and will go to Other
Stroke patient 8 arrived at 5.294334444516364 days and will go to Other
Complex Neuro patient 8 finished treatment at 5.294594652124219 days
Stroke patient 8 finished treatment at 5.602075991683166 days
Stroke patient 8 f

In [ ]:
def llm_lognormal(mean, std):
    '''
    Wrapper for LLM lognormal code for calculating mu, sigma of underlying
    normal distribution.

    The function has been created by the tester. The calculation generated
    by the LLM.

    Params:
    ------
    stay_distribution
    '''
    
    mu = np.log(mean**2 / np.sqrt(mean**2 + std**2))
    sigma = np.sqrt(np.log(1 + (std**2 / mean**2)))
    
    return mu, sigma

In [ ]:
mean, std = 128.79, 267.51
expected_moments = Lognormal(mean, std)
print(expected_moments.mu, expected_moments.sigma)

llm_mu, llm_sigma = llm_lognormal(mean, std)
print(llm_mu, llm_sigma)

(llm_mu, llm_sigma) == (expected_moments.mu, expected_moments.sigma)

## Testing

* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods




### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [ ]:
M = 10_000_000

In [ ]:
def test_1(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = 1.2
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [ ]:
test_1(M)

### Extreme value test 2

* The code is again modified so that all patient typs have a mean inter-arrival rate of $M$

**Result: PASS**

In [ ]:
def test_2(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = large_number
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [ ]:
test_2(M)

### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [ ]:
TEN_DAYS = 10

In [ ]:
def test_3(run_length):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env, run_length=run_length)
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(env.now)
    
